# Landlords and Renters Impacted by BPS

We need to identify potential opposition from owners of large buildings. We should identify:

- Which owners have the largest portfolios, by number of buildings, total buildings sq. ft., and number of units.
- Which owners have the highest GHGE and EUI.
- Which owners have the highest use of each energy source (gas, electricity, and steam).
- How many owners have only one or two buildings? Small landlords are often short-staffed family businesses and may have more trouble arranging for retrofits.

We also need to estimate the number of tenants impacted by BPS. We should identify:

- How many units are impacted by BPS
- An estimate of how many tenants are in these units (using average occupants of a studio, 1 bedroom, 2 bedroom, etc. We can use national statistics, but if we have numbers for Seattle or Washington state, that would be better).

As a stretch goal:
- Can we identify the companies that are owned by a larger company? For example, an LLC may own only one building, but the LLC is owned by Vulcan, which has numerous holdings.

In [3]:
import numpy
# Let's use Levenshtein distance to make our lives easier
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
import pandas_gbq
import pandas

In [5]:
query = """
    SELECT *
    FROM `GHGE_buildings_data.building-benchmark`;
"""

buildings_dataframe = pandas_gbq.read_gbq(query, project_id='seattle-377109') # revisit after we cast the column

Downloading: 100%|█████████████████████████████████████████████████████████████|


In [6]:
buildings_dataframe.head()

,OSEBuildingID,TaxParcelIdentificationNumber,DataYear,BuildingType,BuildingName,Owner,CouncilDistrictCode,Neighborhood,Units,YearBuilt,...,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,Outlier,ComplianceIssue,ComplianceStatus,Comments,DefaultData,LegislationPropertyType
0,50148,7733600135,2020,NonResidential,RESIDENCE INN SEATTLE U,TMUD GSL LLC,4,NORTHEAST,None,2016,...,Parking,54695,None,<NA>,None,No Issue,Compliant,None,None,Hotel
1,50150,1991200090,2020,NonResidential,HYATT HOUSE,HH SEATTLE LLC,1,LAKE UNION,None,2016,...,Parking,28069,None,<NA>,None,No Issue,Compliant,None,None,Hotel
2,50160,659000775,2020,NonResidential,AMAZON DOPPLER BUILDING,ACORN DEVELOPMENT LLC,7,DOWNTOWN,None,2016,...,Parking,448625,Convention Center,59672,None,No Issue,Compliant,None,None,Office
3,50166,4083306985,2020,NonResidential,TABLEAU (NORTH EDGE),BRE-BMR 34TH LLC,1,LAKE UNION,None,2016,...,Parking,117684,Restaurant,2839,None,No Issue,Compliant,None,None,Office
4,50192,660001605,2020,NonResidential,MIDTOWN 21 (AMAZON),MIDTOWN21 LLC,1,DOWNTOWN,None,2016,...,Parking,110813,None,<NA>,None,No Issue,Compliant,None,None,Office


In [7]:
# find owners with largest number of holdings and units
owner_data_frame = buildings_dataframe['Owner'].value_counts().sort_values(ascending=False)
owner_data_frame[:20]

SEATTLE CITY OF                              106
NOT FOUND                                     50
BOARD OF REGENTS OF THE UW                    24
HOUSING AUTHORITY CITY SEATTLE                14
OVERLOOK MAGNOLIA APARTMENTS LLC              11
CSHV NWCP SEATTLE LLC                         10
KING COUNTY                                   10
ACORN DEVELOPMENT LLC                          9
SELIG HOLDINGS COMPANY LLC                     9
SEATTLE HOUSING AUTHORITY                      8
QWEST CORPORATION                              8
SEATTLE SCHOOL DISTRICT NO 1                   8
BREIER SCHEETZ PROPERTIES LLC                  7
PUBLIC STORAGE                                 7
CITY OF SEATTLE                                7
UNIVERSITY VILLAGE LIMITED PS                  6
SREH 2014 LLC                                  6
SEATTLE UNIVERSITY                             6
WASHINGTON STATE OF                            5
UNIVERSITY OF WASHINGTON BOARD OF REGENTS      5
Name: Owner, dtype: 

We have some owners with different names listed. Let's fix that with some Levenshtein distance to help us out.

In [8]:
# methods to normalize some of our owner names

def normalize_owner(old_owner_name, new_owner_name):
    buildings_dataframe['Owner'] = numpy.where(buildings_dataframe['Owner'] == old_owner_name, new_owner_name, buildings_dataframe['Owner'])

def normalize_owners(old_owner_names, new_owner_name):
    buildings_dataframe['Owner'] = numpy.where(buildings_dataframe['Owner'].isin(old_owner_names), new_owner_name, buildings_dataframe['Owner'])

In [9]:
# quick and dirty assessment of the top twenty 
top_twenty_owners = buildings_dataframe['Owner'].value_counts().sort_values(ascending=False)[:20].keys()
all_owners = list(buildings_dataframe['Owner'].unique())

# this gives linguistic similarity between owners
for owner in top_twenty_owners:
    print([owner, process.extract(owner, all_owners, limit=10)], '\n')

['SEATTLE CITY OF', [('SEATTLE CITY OF', 100), ('CITY OF SEATTLE', 95), ('THE CITY OF SEATTLE', 95), ('SEATTLE CITY OF: SDOT', 95), ('SEATTLE OF CITY', 95), ('SEATTLE CITY OF SEATRAN R/W', 90), ('SEATTLE CITY OF/DEPT OF TRANSPORTATION', 90), ('SEATTLE CITY OF % TERRY DUNNING', 90), ('SEATTLE CITY OF S-DOT R/W', 90), ('SEATTLE CITY OF ATTN DHHS DIRECTOR', 90)]] 

['NOT FOUND', [('NOT FOUND', 100), ('INTER CO-OP USA NOT 7 ASSOCIATES PARTNERSHIP', 86), ('SOUND', 72), ('POCOCK ROWING FOUNDATION', 70), ('GEE HOW OAK TIN FOUNDATION', 70), ('BILL & MELINDA GATES FOUNDATION', 70), ('SOUTHEAST SEATTLE SENIOR FOUNDATION', 70), ('CENTRAL PUGET SOUND REGIONAL TRANSIT AUTHORITY', 60), ('SEATTLE FUNDING CORP', 60), ('PUGET SOUND ENERGY INC', 60)]] 

['BOARD OF REGENTS OF THE UW', [('BOARD OF REGENTS OF THE UW', 100), ('SEATTLE CITY OF', 86), ('UNIVERSITY OF WASHINGTON BOARD OF REGENTS', 86), ('EXPEDITORS INTERNATIONAL OF WASHINGTON INC', 86), ('WASHINGTON STATE OF: DEPARTMENT OF TRANSPORTATION', 86)

The city of Seattle, state of Washington, etc. have multiple departments. Let's add a method to list owner and department separately. For our general use case, we only need to know it's owned by UW/the city/the state, but at some point in the future it may be useful to have the departments.

In [10]:
def normalize_owners_and_dept(old_owner_names, new_owner_name, dept):
    buildings_dataframe['Dept'] = numpy.where(buildings_dataframe['Owner'].isin(old_owner_names), dept, buildings_dataframe['Dept'])
    buildings_dataframe['Owner'] = numpy.where(buildings_dataframe['Owner'].isin(old_owner_names), new_owner_name, buildings_dataframe['Owner'])

In [13]:
buildings_dataframe['Dept'] = numpy.nan

In [14]:
# Normalize City of Seattle DOT

normalize_owners_and_dept([
    'SEATTLE CITY OF: SDOT', 
    'SEATTLE CITY OF/DEPT OF TRANSPORTATION', 
    'SEATTLE CITY OF SEATRAN R/W',
    'SEATTLE CITY OF S-DOT R/W'
], 'CITY OF SEATTLE', 'TRANSPORTATION')

buildings_dataframe[buildings_dataframe['Dept'] == 'TRANSPORTATION']

,OSEBuildingID,TaxParcelIdentificationNumber,DataYear,BuildingType,BuildingName,Owner,CouncilDistrictCode,Neighborhood,Units,YearBuilt,...,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,Outlier,ComplianceIssue,ComplianceStatus,Comments,DefaultData,LegislationPropertyType,Dept
251,19841,660001875,2020,Multifamily MR (5-9),VILLA APTS,CITY OF SEATTLE,7,EAST,62,1908,...,7865,Parking,4385,None,No Issue,Compliant,None,None,Multifamily Housing,TRANSPORTATION
531,601,4083300140,2020,NonResidential,WALLINGFORD CENTER,CITY OF SEATTLE,4,LAKE UNION,NOT_FOUND,1904,...,27427,Multifamily Housing,18130,None,No Issue,Compliant,None,None,Mixed Use Property,TRANSPORTATION
1033,26409,8816400420,2020,Multifamily LR (1-4),DAVISON APTS - 22 UNITS,CITY OF SEATTLE,4,NORTHEAST,22,1926,...,1416,None,<NA>,None,No Issue,Compliant,None,None,Multifamily Housing,TRANSPORTATION
1715,50019,1989201221,2020,Multifamily MR (5-9),306 QUEENE ANNE APTS,CITY OF SEATTLE,7,MAGNOLIA / QUEEN ANNE,None,2014,...,3389,None,<NA>,None,No Issue,Compliant,None,None,Multifamily Housing,TRANSPORTATION
2080,20407,945000680,2020,Multifamily LR (1-4),CATE APTS,CITY OF SEATTLE,1,NORTHWEST,31,2003,...,7400,Office,1005,None,No Issue,Compliant,None,None,Multifamily Housing,TRANSPORTATION


In [15]:
buildings_dataframe[buildings_dataframe['Owner'] == 'CITY OF SEATTLE'].Dept.unique()

array(['TRANSPORTATION', 'nan'], dtype=object)

In [16]:
# Seattle Housing Authority

normalize_owners_and_dept([
    'HOUSING AUTHORITY CITY SEATTLE', 
    'SEATTLE HOUSING AUTHORITY', 
    'HOUSING AUTHORITY OF THE CITY OF SEATTLE', 
    'HOUSING AUTHORITY OF CITY OF SEATTLE', 
    'THE HOUSING AUTHORITY OF THE CITY OF SEATTLE', 
    'SEATTLE CITY OF HOUSING AUTHORITY', 
    'HOUSING AUTHORITY OF SEATTLE'
], 'CITY OF SEATTLE', 'HOUSING')

In [17]:
# City of Seattle 

normalize_owners([
    'SEATTLE CITY OF',
    'THE CITY OF SEATTLE',
    'SEATTLE OF CITY',
    'SEATTLE CITY OF % TERRY DUNNING' # Terry Dunning is a former City of Seattle employee
], 'CITY OF SEATTLE')

In [18]:
# Normalize UW

normalize_owners([
    'BOARD OF REGENTS OF THE UW',
    'UNIVERSITY OF WASHINGTON BOARD OF REGENTS', 
    'WASHINGTON UNIVERSITY OF BOARD OF REGENTS THE',
    'WASHINGTON UNIVERSITY OF BOARD OF REGENTS', 
    'BOARD OF REGENTS OF THE UNIVERSITY OF WASHINGTON'
], 'UNIVERSITY OF WASHINGTON')

In [19]:
# King County

normalize_owners(['KING-COUNTY GOVT'], 'KING COUNTY')

normalize_owners_and_dept(['KING COUNTY - WTD'], 'KING COUNTY', 'TRANSPORTATION')

normalize_owners_and_dept(['KING COUNTY HOUSING AUTHORITY'], 'KING COUNTY', 'HOUSING')

normalize_owners_and_dept(['KING COUNTY: DEPT OF EXECUTIVE SERVICES/REAL ESTATE SERIVCES-FACILITIES MANAGEMENT DIV'], 'KING COUNTY', 'FACILITIES')

In [20]:
# Qwest (stadium owners)

normalize_owners(['QWEST CORPORATION', 'QWEST CORP'], 'QWEST CORPORATION')

In [21]:
# Seattle Schools

normalize_owners_and_dept([
    'SEATTLE SCHOOL DISTRICT NO 1',
    'SEATTLE SCHOOL DISTRICT #1'
], 'CITY OF SEATTLE', 'SEATTLE SCHOOL DISTRICT NO 1')

In [22]:
# Public Storage (privately held company, despite the name)
normalize_owners([ 
    'PUBLIC STORAGE PROPERTIES VII',
    'PUBLIC STORAGE PROP XV INC', 
    'PUBLIC STORAGE PROP XVI INC'
], 'PUBLIC STORAGE')

In [23]:
normalize_owners([
    'WASHINGTON STATE OF', 
    'WASHINGTON STATE GOVT', 
    'WASHINGTON STATE-GOVT'
], 'STATE OF WASHINGTON')

normalize_owners_and_dept([
    'WASHINGTON STATE OF: DEPARTMENT OF TRANSPORTATION',
    'WASHINGTON STATE OF-DEPARTMENT OF TRANSPORTATION',
    'WASHINGTON STATE OF/DEPARTMENT OF TRANSPORTATION',
    'WASHINGTON STATE -DOT'
], 'STATE OF WASHINGTON', 'TRANSPORTATION')

normalize_owners_and_dept([
    'WASHINGTON STATE OF-DEPT OF COMMERCE'
], 'STATE OF WASHINGTON', 'COMMERCE')


Are there matches with:
- UW?
- King County?
- Archdiocese?
- Other Seattle City departments?
- State of Washington? 
- Seattle schools?
- storage companies?
    

In [26]:
# See if there are UW buildings hiding

# fix NA error
buildings_dataframe.fillna({'Owner':'UNDEFINED'},  inplace=True)

buildings_dataframe[buildings_dataframe['Owner'].str.contains('UNIV')].Owner.unique()

array(['UNIVERSITY OF WASHINGTON', 'TEN-TWENTY UNIVERSITY L L C',
       'UNIVERSITY HOUSE AT WALLINGFORD L L C',
       'UNIVERSITY SILVER CLOUD INN', 'JLW UNIVERSITY INVESTMENTS LLC',
       'UNIVERSITY VILLAGE LIMITED PS', 'UNIVERSITY CENTER LLC',
       'RAINIER NORTHWEST-UNIVERSITY LLC',
       'SEATTLE UNIVERSITY DISTRICT LP', 'GRE UNIVERSITY L L C',
       'UNIVERISTY PARK INVESTMENTS L', 'UNIVERSITY PROPERTY X LLC',
       'MG AT UNIVERSITY LLC',
       'UNIVERSITY HOLDINGS 1 LLC +UNIVERSITY HOLDINGS 2 LLC +ET AL',
       '5200 UNIVERSITY WAY LLC', 'UNIVERSITY OF HAWAII FOUNDATION',
       'UNIVERSITY PREPARATORY ACADEMY', 'SEATTLE PACIFIC UNIVERSITY',
       'SEATTLE UNIVERSITY'], dtype=object)

[University House at Wallingford](https://www.eraliving.com/communities/wallingford/) is a retirement home that has a "relationship" with the [University of Washington Retirement Assocation](https://www.washington.edu/uwra/). They have a second facility in Issaquah. Both are owned by [Era Retirement Living](https://www.eraliving.com/), which owns a total of eight retirement homes in the greater Seattle area.

I looked up the other "university" owner names, no connections with UW

In [27]:
# See if there are other UW buildings hiding
buildings_dataframe[buildings_dataframe['Owner'].str.contains('UW')].Owner.unique()

array(["B'NAI B'RITH HILLEL FNTDTN AT THE UW", 'UW 20TH AVE LLC'],
      dtype=object)

Hillel is not owned by UW, it's a 501c3.

In [28]:
buildings_dataframe[buildings_dataframe['Owner'] == 'UW 20TH AVE LLC']

,OSEBuildingID,TaxParcelIdentificationNumber,DataYear,BuildingType,BuildingName,Owner,CouncilDistrictCode,Neighborhood,Units,YearBuilt,...,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,Outlier,ComplianceIssue,ComplianceStatus,Comments,DefaultData,LegislationPropertyType,Dept
1053,50491,925049120,2020,Multifamily LR (1-4),U20 APARTMENTS,UW 20TH AVE LLC,1,NORTHEAST,None,2019,...,<NA>,None,<NA>,None,No Issue,Compliant,None,None,Multifamily Housing,nan


UW 20th Ave LLC is an apartment complex, unaffiliated with UW.

In [30]:
# See if there are King County related companies
print(buildings_dataframe[buildings_dataframe['Owner'].str.contains('KING')].Owner.unique())
buildings_dataframe[buildings_dataframe['Owner'].str.contains('COUNTY')].Owner.unique()

['KING COUNTY' 'ONE HUNDRED KING L L C' 'HUDSON FIRST & KING LLC'
 'MSI - 1ST & KING LLC' 'GATEWAY KING LLC' '255 SOUTH KING STREET LP'
 'MJWKING LLC' 'HISTORICAL SOCIETY OF SEATTLE & KING COUNTY'
 'BOYS & GIRLS CLUBS OF KING COUNTY `' 'YEE KING G LLC'
 '5940 MARTIN LUTHER KING WAY SOUTH LLC' 'DIAMOND PARKING INC']


array(['KING COUNTY', 'HISTORICAL SOCIETY OF SEATTLE & KING COUNTY',
       'BOYS & GIRLS CLUBS OF KING COUNTY `'], dtype=object)

Looks like there aren't. Let's look at the Archdiosece.

In [31]:
# What buildings are associated with the Catholic Archdiosece?

print(buildings_dataframe[buildings_dataframe['Owner'].str.contains('ARCH')].Owner.unique())
buildings_dataframe[buildings_dataframe['Owner'].str.contains('CATH')].Owner.unique()

['VIRGINIA MASON RESEARCH CENTER'
 'ARCHDIOCESAN HOUSING AUTHORITY C/O CATHOLIC HOUSING SERVICES OF WESTERN WASHINGTON'
 'CATHOLIC ARCHBISHOP OF SEATTLE CORPORATION OF THE'
 'CORPORATION CATHOLIC ARCHBISHOP SEATTLE'
 'FRED HUTCHINSON CANCER RESEARCH CTR'
 'FRED HUTCHINSON CANCER RESEARCH CENTER' 'GATES ARCHIVE LLC'
 'FRED HUTCHISON CANCER RESEARCH CENTER' 'ARCHDIOCESAN HOUSING AUTHORITY'
 'PACIFIC NORTHWEST RESEARCH' 'CATHOLIC ARCHBISHOP CORP OF']


array(['ASSOCIATED CATHOLIC CEMETERIES',
       'ARCHDIOCESAN HOUSING AUTHORITY C/O CATHOLIC HOUSING SERVICES OF WESTERN WASHINGTON',
       'CATHOLIC ARCHBISHOP OF SEATTLE CORPORATION OF THE',
       'CORPORATION CATHOLIC ARCHBISHOP SEATTLE', 'CATHAY INVESTMENT LLC',
       'CATHOLIC ARCHBISHOP CORP OF', 'CATHOLIC COMMUNITY SRVCS'],
      dtype=object)

In [32]:
normalize_owners([
    'CATHOLIC ARCHBISHOP OF SEATTLE CORPORATION OF THE',
    'CORPORATION CATHOLIC ARCHBISHOP SEATTLE',
    'CATHOLIC ARCHBISHOP CORP OF'
], 'CATHOLIC ARCHDIOCESE OF SEATTLE')

normalize_owners_and_dept([
    'ARCHDIOCESAN HOUSING AUTHORITY C/O CATHOLIC HOUSING SERVICES OF WESTERN WASHINGTON',
    'ARCHDIOCESAN HOUSING AUTHORITY'
], 'CATHOLIC ARCHDIOCESE OF SEATTLE', 'HOUSING SERVICES')

normalize_owners_and_dept([
    'ASSOCIATED CATHOLIC CEMETERIES',
], 'CATHOLIC ARCHDIOCESE OF SEATTLE', 'CEMETARIES')

normalize_owners_and_dept([
    'CATHOLIC COMMUNITY SRVCS'
], 'CATHOLIC ARCHDIOCESE OF SEATTLE', 'COMMUNITY SERVICES')

In [33]:
# Since we're here, what about Fred Hutch?
print(buildings_dataframe[buildings_dataframe['Owner'].str.contains('FRED')].Owner.unique())
buildings_dataframe[buildings_dataframe['Owner'].str.contains('HUTCH')].Owner.unique()

['FRED HUTCHINSON CANCER RESEARCH CTR'
 'FRED HUTCHINSON CANCER RESEARCH CENTER' 'FRED HUTCHINSON CANCER RES CTR'
 'FRED HUTCHISON CANCER RESEARCH CENTER' 'FRED MEYER STORES INC']


array(['FRED HUTCHINSON CANCER RESEARCH CTR',
       'FRED HUTCHINSON CANCER RESEARCH CENTER',
       'FRED HUTCHINSON CANCER RES CTR',
       'FRED HUTCHISON CANCER RESEARCH CENTER', 'HUTCHINSON CAROL S'],
      dtype=object)

In [34]:
normalize_owners([
 'FRED HUTCHINSON CANCER RESEARCH CTR',
 'FRED HUTCHINSON CANCER RESEARCH CENTER',
 'FRED HUTCHINSON CANCER RES CTR'
], 'FRED HUTCHISON CANCER RESEARCH CENTER')

In [35]:
# City of Seattle departments
buildings_dataframe[buildings_dataframe['Owner'].str.contains('CITY')].Owner.unique()

array(['CITY OF SEATTLE',
       'SOVEREIGN ASSOCIATES LLC C/O CITY CENTER REAL ESTATE',
       'SEATTLE-CITY GOVERNMENT', 'CRE QUEEN CITY LLC',
       'HOUSING AUTHORITY OF CITY OF S', 'CITY INVESTORS XII L L C',
       'CITY INVESTORS XXII LLC', 'CITY PLACE IV L L C',
       'DEXTER CITY LLC', 'CITY INVESTORS XXIV LLC',
       'CITY INVESTORS XXX LLC', 'SEATTLE CITY OF ATTN DHHS DIRECTOR',
       'SEATTLE CITY OF-SEATRAN R/W', 'SHORELINE CITY OF',
       'WORKSHOPS SEATTLE LAKE CITY WAY',
       '3030 LAKE CITY LLC A DELAWARE LLC', 'PRE LAKE CITY 26 LLC',
       'LAKE CITY ASSOCIATES L L C',
       'HOUSING AUTHRITY OF THE SEATTLE CITY GOVT',
       'FAIRFIELD LAKE CITY LP', 'FSC LAKE CITY ASSOCIATES LLC',
       'LAKE CITY WAY 137 ASSOCIATES LLC', 'PROMENADE LAKE CITY LLC',
       'LAKE CITY PROJECT ASSOCIATES',
       '13355 LAKE CITY WAY NE ASSOCIATES', 'LAKE CITY 5 LLC',
       'SEATTLE CITY OF: C/O SEATTLE CITY LIGHT',
       'RISE PROPERTIES ( CITYWALK)  LP', 'JUKEBOX CITY BUIL

In [36]:
normalize_owners([
    'SEATTLE-CITY GOVERNMENT'
], 'CITY OF SEATTLE')

normalize_owners_and_dept([
    'HOUSING AUTHORITY OF CITY OF S',
    'HOUSING AUTHRITY OF THE SEATTLE CITY GOVT'
], 'CITY OF SEATTLE', 'HOUSING')

normalize_owners_and_dept([
    'SEATTLE CITY OF-SEATRAN R/W'
], 'CITY OF SEATTLE', 'TRANSPORTATION')

normalize_owners_and_dept([
   'SEATTLE CITY OF ATTN DHHS DIRECTOR'
], 'CITY OF SEATTLE', 'HEALTH AND HUMAN SERVICES')

normalize_owners_and_dept([
   'SEATTLE CITY OF: C/O SEATTLE CITY LIGHT'
], 'CITY OF SEATTLE', 'CITY LIGHT')

normalize_owners_and_dept([
   'SEATTLE CITY OF ENGINEERING DE'
], 'CITY OF SEATTLE', 'ENGINEERING')

In [40]:
# Indigenous-owned/run orgs?
print(buildings_dataframe[buildings_dataframe['Owner'].str.contains('NATIVE')].Owner.unique())
print(buildings_dataframe[buildings_dataframe['Owner'].str.contains('DUWAMISH')].Owner.unique())
buildings_dataframe[buildings_dataframe['Owner'].str.contains('INDIAN')].Owner.unique()

[]
[]


array(['SEATTLE INDIAN SERVICES COMMIS', 'SEATTLE INDIAN HEALTH BOARD'],
      dtype=object)

In [41]:
normalize_owners_and_dept([
    'SEATTLE INDIAN SERVICES COMMIS'
], 'CITY OF SEATTLE', 'INDIAN SERVICES COMMISSION')

The Seattle Indian Health Board and the Seattle Indian Services Commission's buildings fall under BPS. The Indian Services Commission is part of the city government; Seattle Indian Health Board is an independent non-profit. Given 350 Seattle's history of partnering with Indigenous communities, we should be reaching out to both of these organizations.

The Duwamish Longhouse is not covered by BPS.

In [42]:
# State of Washington

# we previously searched for 'WASH'
buildings_dataframe[buildings_dataframe['Owner'].str.contains('STATE')].Owner.unique()

array(['SFERS REAL ESTATE CORP FF', 'STATE OF WASHINGTON',
       'WASHINGTON STATE CONVENTION CENTER',
       'SOVEREIGN ASSOCIATES LLC C/O CITY CENTER REAL ESTATE',
       'WASHINGTON REAL ESTATE HOLDINGS LLC',
       'ELLENBERT APARTMENTS REAL ESTATE INVESTMENTS LLC',
       'BROADWAY ESTATES LLC', 'BURKE-STATE BLDG LLC',
       'WASHINGTON STATE-DEPT OF TRANSPORTATION',
       'UNITED STATES OF AMERICA', 'STATE HOTEL LLC',
       'PACNORTH PROPERTIES LLC+RIVERCROFT REAL ESTATE ASSETS LLC',
       'GEORGETOWN REAL ESTATE LLC',
       'WASHINGTON STATE BOARD FOR COMMUNITY AND TECHNICAL COLLEGE',
       'SOUTH ALASKA STREET REAL ESTATE INVESTMENTS LLC',
       'ESTATE OF REX C OTT +REX C',
       'H5 CAPITAL - SEATTLE REAL ESTATE II LLC',
       'UNION INVESTMENT REAL ESTATE GMBH', 'ACI REAL ESTATE SPE 137 LLC',
       'ELLIOTT SEATTLE REAL ESTATE HOLDING INC',
       'SELIG REAL ESTATE HOLDINGS TEN',
       'KAPLAN REAL ESTATE SERVICES LLC',
       'AUBOMBOLI ITALIANE REAL ESTATE INV

In [43]:
# 'WASHINGTON STATE CONVENTION CENTER'--privately owned
normalize_owners_and_dept([
    'WASHINGTON STATE-DEPT OF TRANSPORTATION'
], 'STATE OF WASHINGTON', 'TRANSPORTATION')

normalize_owners_and_dept([
    'WASHINGTON STATE BOARD FOR COMMUNITY AND TECHNICAL COLLEGE'
], 'STATE OF WASHINGTON', 'COMMUNITY AND TECHNICAL COLLEGEs')

# the parcel data from King County must be wrong: 
# https://blue.kingcounty.com/Assessor/eRealProperty/Detail.aspx?ParcelNbr=5247800715
buildings_dataframe[buildings_dataframe['Owner'] == 'UNITED STATES OF AMERICA']

,OSEBuildingID,TaxParcelIdentificationNumber,DataYear,BuildingType,BuildingName,Owner,CouncilDistrictCode,Neighborhood,Units,YearBuilt,...,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,Outlier,ComplianceIssue,ComplianceStatus,Comments,DefaultData,LegislationPropertyType,Dept
786,26600,5247800715,2020,NonResidential,CADILLAC HOTEL/HISTORICAL SOCIETY,UNITED STATES OF AMERICA,7,DOWNTOWN,NOT_FOUND,1966,...,13000,None,<NA>,None,No Issue,Compliant,None,None,Mixed Use Property,nan


In [44]:
buildings_dataframe[buildings_dataframe['Owner'].str.contains('SCHOOL')].Owner.unique()

array(['SCHOOL BISHOP BLANCHET HIGH', 'LUTHERAN HIGH SCHOOL ASSN WASH',
       'EPIPHANY SCHOOL', 'NORTHWEST SCHOOL OF THE ARTS',
       'SEATTLE SCHOOL DISTRICT NUMBER 1', 'BILLINGS MIDDLE SCHOOL',
       'HAMLIN ROBINSON SCHOOL', 'WESTSIDE SCHOOL'], dtype=object)

In [45]:
normalize_owners_and_dept([
    'SEATTLE SCHOOL DISTRICT NUMBER 1'
], 'CITY OF SEATTLE', 'SEATTLE SCHOOL DISTRICT NO 1')

In [46]:
# Storage companies 

buildings_dataframe[buildings_dataframe['Owner'].str.contains('STORAGE')].Owner.unique()
# hooray, nothing to do!

array(['PUBLIC STORAGE', 'GREENLAKE SELF STORAGE LLC',
       'METRO SELF STORAGE', 'ROOSEVELT ECONO-MINI STORAGE LLC',
       'MAGNOLIA BRIDGE SELF STORAGE INVESTMENT LLC',
       'A-1 SELF STORAGE LLC', 'INTERBAY URBAN STORAGE LLC',
       'STORAGE EQUITIES INC', 'NORTHGATE STORAGE LLC',
       'TWENTY-FOUR SAC SELF-STORAGE LP'], dtype=object)

In [47]:
# Potential UW Shell companies
# This article has a map of UW-owned properties in downtown: https://www.dailyuw.com/news/in-a-housing-crisis-what-is-the-uw-s-role-as-a-landlord/article_5cd3d55e-0c08-11ea-96fa-77e4a1f0cc4c.html
# Are any owned by shell companies?

buildings_dataframe[buildings_dataframe['Owner'] == 'UNIVERSITY OF WASHINGTON'].Address

53            1959 NE PACIFIC ST
100                 1326 5TH AVE
101                 1200 5TH AVE
102                 1325 4TH AVE
136                 1215 4TH AVE
203                 1326 5TH AVE
917             4311 12TH AVE NE
920              1413 NE 45TH ST
921         4333 BROOKLYN AVE NE
938             4310 12TH AVE NE
959        4245 ROOSEVELT WAY NE
963        3800 MONTLAKE BLVD NE
972         3931 BROOKLYN AVE NE
973             4000 15TH AVE NE
974             4000 15TH AVE NE
975             4000 15TH AVE NE
976             4000 15TH AVE NE
977             3900 15TH AVE NE
978        4225 ROOSEVELT WAY NE
1090         1101 NE CAMPUS PKWY
1108          1315 NE CAMPUS PKY
1109             1112 NE 41ST ST
1110          1201 NE CAMPUS PKY
1111          1009 NE PACIFIC ST
3071             1536 N 115TH ST
3072        10330 MERIDIAN AVE N
3074             1570 N 115TH ST
3167             1550 N 115TH ST
3509      3701 Walla Walla Rd NE
3510    3946 WEST STEVENS WAY NE
3511    37

Crossreferencing the addresses and the map in the [article about downtown UW properties](https://www.dailyuw.com/news/in-a-housing-crisis-what-is-the-uw-s-role-as-a-landlord/article_5cd3d55e-0c08-11ea-96fa-77e4a1f0cc4c.html), UW does not appear in our data as the owner of the Fairmont Olympic or the Rainier Tower/Seattle Chamber of Commerce.

Let's see what we can find.

For the Rainier Tower building, there is one building parcel and four parcels for the individual units [here](https://blue.kingcounty.com/Assessor/eRealProperty/Detail.aspx?ParcelNbr=7137830010) are the property details for one unit from the King County Assessor (they all have the same owner). The building is parcel 7137830000. The units within it (Units A-B) are parcels 7137830010, 7137830020, 7137830030, 7137830040.

We can see that the building name is University of Washington but the last purchase was by WRC 4th Avenue LLC. What is WRC LLC?

Suggest searching 'UNIVERSITY OF WASHINGTON' in the name field on the [Dept. of Assessments parcel finder](https://blue.kingcounty.com/Assessor/eRealProperty/default.aspx) to identify potentially missing buildings. Will leave this to other organizers who are more familiar with UW properties.

The parcels in the Fairmont Olympic building seem to be labeled as being part of the parcels in the Rainier Tower building, e.g. [this parcel](https://blue.kingcounty.com/Assessor/eRealProperty/Dashboard.aspx?ParcelNbr=7137830000).

In [48]:
# Which buildings are in our dataset?

UW_building_parcel_ids = [7137830000, 137830010, 7137830020, 7137830030, 7137830040]
for parcel in UW_building_parcel_ids:
    print(parcel)
    print(len(buildings_dataframe[buildings_dataframe['TaxParcelIdentificationNumber'] == str(parcel)]))

7137830000
1
137830010
0
7137830020
0
7137830030
0
7137830040
1


In [49]:
# Set owners for UW buildings downtown

confirmed_UW_buildings = ['7137830000', '7137830040']

for parcel in confirmed_UW_buildings:
    buildings_dataframe['Owner'] = numpy.where(buildings_dataframe['TaxParcelIdentificationNumber'] == parcel, 'UNIVERSITY OF WASHINGTON', buildings_dataframe['Owner'])


In [93]:
# Output UW buildings to its own CSV

buildings_dataframe[buildings_dataframe['Owner'] == 'UNIVERSITY OF WASHINGTON'].to_csv('university_of_washington_buildings.csv')

In [50]:
# create CSV of updated owners so we have it saved

buildings_dataframe.to_csv('updated_owners_2_15_23.csv')

# Analysis

What questions do we want to ask?

Recap from top of notebook:

- Which owners have the largest portfolios, by number of buildings, total buildings sq. ft., and number of units.
- How much of the city's pollution do each of them account for? Do any of them produce more/less pollution than we would expect? (Total pollution / sq ft = average EUI)
- What fuels do the biggest polluters/landowners use the most?
- Which owners have the highest GHGE and EUI.
- Which owners have the highest use of each energy source (gas, electricity, and steam).

We also need to estimate the number of tenants impacted by BPS. We should identify:

- How many units are impacted by BPS
- An estimate of how many tenants are in these units (using average occupants of a studio, 1 bedroom, 2 bedroom, etc. We can use national statistics, but if we have numbers for Seattle or Washington state, that would be better).


In [56]:
# 20 largest landowners by number of buildings owned

aggregate_owners = buildings_dataframe[~buildings_dataframe['Owner'].isin(['UNDEFINED', 'NOT FOUND'])]['Owner'].value_counts(ascending=False).rename_axis('Owner').reset_index(name='BuildingsOwned')
aggregate_owners[:20]

,Owner,BuildingsOwned
0,CITY OF SEATTLE,175
1,UNIVERSITY OF WASHINGTON,43
2,STATE OF WASHINGTON,17
3,KING COUNTY,16
4,OVERLOOK MAGNOLIA APARTMENTS LLC,11
5,CATHOLIC ARCHDIOCESE OF SEATTLE,11
6,CSHV NWCP SEATTLE LLC,10
7,QWEST CORPORATION,10
8,PUBLIC STORAGE,10
9,SELIG HOLDINGS COMPANY LLC,9


In [55]:
# normalize not found values
buildings_dataframe['Owner'] == buildings_dataframe['Owner'].replace('UNDEFINED', 'NOT FOUND', inplace=True)

0       False
1       False
2       False
3       False
4       False
        ...  
3584    False
3585    False
3586    False
3587    False
3588    False
Name: Owner, Length: 3589, dtype: bool

In [126]:
type(buildings_dataframe['PropertyGFABuilding_s_'][1])

numpy.int64

In [131]:
# create temp df with owners + total sq ft

# left join on owner name w/ aggregate owners
#aggregate_owners = buildings_dataframe[~buildings_dataframe['Owner'].isin(['UNDEFINED', 'NOT FOUND'])]['Owner'].value_counts(ascending=False).rename_axis('Owner').reset_index(name='BuildingsOwned')
temp_total_sq_ft = buildings_dataframe.groupby(by='Owner')['PropertyGFABuilding_s_'].sum().rename_axis('Owner').reset_index(name='TotalSquareFootage')
temp_total_sq_ft.head()

,Owner,TotalSquareFootage
0,1 BOND TOWER LLC,93660
1,100 ALOHA LP,43172
2,1000 1ST AVENUE SOUTH LIMITED PARTNERSHIP,64800
3,1000 MADISON LLC,51900
4,1000-1100 DEXTER AVENUE INC,234236


In [101]:
a = aggregate_owners.astype({ 'Owner': 'str' })
a.dtypes

Owner                 object
BuildingsOwned         int64
TotalSquareFootage     Int64
dtype: object

In [100]:
temp_total_sq_ft.dtypes

Owner                 object
TotalSquareFootage     Int64
dtype: object

We need to convert the Numpy type Int64 to int64 in TotalSquareFootage. Otherwise we'll return NaN.

In [124]:
aggregate_owners = aggregate_owners.drop('TotalSquareFootage', axis=1)

In [105]:
aggregate_owners['Owner'].isin(temp_total_sq_ft['Owner']).value_counts()

True    2078
Name: Owner, dtype: int64

In [114]:
temp_total_sq_ft = temp_total_sq_ft.astype({ 'TotalSquareFootage': 'int64' })
temp_total_sq_ft.dtypes

Owner                 object
TotalSquareFootage     int64
dtype: object

In [125]:
aggregate_owners.dtypes

Owner             object
BuildingsOwned     int64
dtype: object

Success! All numbers are int64 types!

In [128]:
aggregate_owners = aggregate_owners.merge(temp_total_sq_ft, left_on='Owner', right_on='Owner', how='left')
aggregate_owners.head()

,Owner,BuildingsOwned,TotalSquareFootage
0,CITY OF SEATTLE,175,17038324
1,UNIVERSITY OF WASHINGTON,43,18445870
2,STATE OF WASHINGTON,17,2768746
3,KING COUNTY,16,4355059
4,OVERLOOK MAGNOLIA APARTMENTS LLC,11,388815


In [182]:
def add_aggregate_vals(main_df, new_col_name, col_to_aggregate, data_type):
    temp_col = buildings_dataframe.groupby(by='Owner')[col_to_aggregate].sum().rename_axis('Owner').reset_index(name=new_col_name).astype({ new_col_name: data_type })
    temp_df = main_df.merge(temp_col, left_on='Owner', right_on='Owner', how='left')
    return temp_df

In [141]:
# add total GHGE

aggregate_owners = add_aggregate_vals(aggregate_owners, 'TotalGHGEMetricTons', 'TotalGHGEmissions__metric_tons_', 'int64')

aggregate_owners.head()

,Owner,BuildingsOwned,TotalSquareFootage,TotalGHGEmissions__metric_tons_
0,CITY OF SEATTLE,175,17038324,16357
1,UNIVERSITY OF WASHINGTON,43,18445870,39712
2,STATE OF WASHINGTON,17,2768746,3614
3,KING COUNTY,16,4355059,22535
4,OVERLOOK MAGNOLIA APARTMENTS LLC,11,388815,52


In [148]:
cols_to_add = [
    [aggregate_owners, 'TotalNaturalGas_kBtu', 'NaturalGas_kBtu_', 'int64'],
    [aggregate_owners, 'TotalElectricity_kBtu', 'Electricity_kBtu_', 'int64'],
    [aggregate_owners, 'TotalSteamUse_kBtu', 'SteamUse_kBtu_', 'int64'],
    [aggregate_owners, 'TotalOtherFuelUse_kBtu', 'OtherFuelUse_kBtu_', 'int64']
]

aggregate_owners = add_aggregate_vals(*cols_to_add[0])
aggregate_owners.head()

,Owner,BuildingsOwned,TotalSquareFootage,TotalGHGEmissions__metric_tons_,NaturalGas_kBtu_
0,CITY OF SEATTLE,175,17038324,16357,247129120
1,UNIVERSITY OF WASHINGTON,43,18445870,39712,106707401
2,STATE OF WASHINGTON,17,2768746,3614,59208493
3,KING COUNTY,16,4355059,22535,372904431
4,OVERLOOK MAGNOLIA APARTMENTS LLC,11,388815,52,0


In [155]:
# for col in cols_to_add[1:]:
#     aggregate_owners = add_aggregate_vals(*col)

aggregate_owners = add_aggregate_vals(aggregate_owners, 'TotalElectricity_kBtu', 'Electricity_kBtu_', 'int64')
aggregate_owners.head()

KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'Electricity_kBtu_' not found in columns."

In [167]:
# let's debug
temp_col = buildings_dataframe.groupby(by='Owner')['Electricity_kBtu_'].sum().rename_axis('Owner')
temp_col = temp_col.reset_index(name='TotalElectricity_kBtu')
temp_col = temp_col.astype({ 'TotalElectricity_kBtu': 'int64' }) # issue was with key in this dict--needed to be new col name, not old

In [172]:
# this doesn't add columns, not sure why
for col in cols_to_add[1:]:
    aggregate_owners = add_aggregate_vals(*col)
    aggregate_owners.head()

In [193]:
temp_col = buildings_dataframe.groupby(by='Owner')['Electricity_kBtu_'].sum().rename_axis('Owner').reset_index(name='TotalElectricity_kBtu').astype({ 'TotalElectricity_kBtu': 'int64' })
temp_col.head()
temp_df = aggregate_owners.merge(temp_col, left_on='Owner', right_on='Owner', how='left')
temp_df.head()
aggregate_owners = temp_df
aggregate_owners.head()

,Owner,BuildingsOwned,TotalSquareFootage,TotalGHGEmissions__metric_tons_,TotalElectricity_kBtu_x,TotalElectricity_kBtu_y
0,CITY OF SEATTLE,175,17038324,16357,501287744,501287744
1,UNIVERSITY OF WASHINGTON,43,18445870,39712,679730875,679730875
2,STATE OF WASHINGTON,17,2768746,3614,88426130,88426130
3,KING COUNTY,16,4355059,22535,255701627,255701627
4,OVERLOOK MAGNOLIA APARTMENTS LLC,11,388815,52,9911319,9911319


In [196]:
aggregate_owners = aggregate_owners.drop('TotalElectricity_kBtu_y', axis=1)
list(aggregate_owners)

['Owner',
 'BuildingsOwned',
 'TotalSquareFootage',
 'TotalGHGEmissions__metric_tons_',
 'TotalElectricity_kBtu_x']

In [197]:
# doing this the quick and dirty way
aggregate_owners = add_aggregate_vals(aggregate_owners, 'TotalSteamUse_kBtu', 'SteamUse_kBtu_', 'int64')

In [200]:
aggregate_owners = add_aggregate_vals(aggregate_owners, 'TotalOtherFuelUse_kBtu', 'OtherFuelUse_kBtu_', 'int64')
aggregate_owners.head()

,Owner,BuildingsOwned,TotalSquareFootage,TotalGHGEmissions__metric_tons_,TotalElectricity_kBtu_x,TotalSteamUse_kBtu,TotalNaturalGas_kBtu,TotalOtherFuelUse_kBtu
0,CITY OF SEATTLE,175,17038324,16357,501287744,6826192,247129120,0
1,UNIVERSITY OF WASHINGTON,43,18445870,39712,679730875,366595771,106707401,0
2,STATE OF WASHINGTON,17,2768746,3614,88426130,0,59208493,0
3,KING COUNTY,16,4355059,22535,255701627,16515088,372904431,0
4,OVERLOOK MAGNOLIA APARTMENTS LLC,11,388815,52,9911319,0,0,0


In [ ]:
aggregate_owners = add_aggregate_vals(aggregate_owners, 'TotalNaturalGas_kBtu', 'NaturalGas_kBtu_', 'int64')
aggregate_owners.head()

In [202]:
aggregate_owners = aggregate_owners.rename(columns={'TotalElectricity_kBtu_x': 'TotalElectricity_kBtu_'})

In [206]:
def add_avg_vals(main_df, new_col_name, col_to_avg):
    temp_col = buildings_dataframe.groupby(by='Owner')[col_to_avg].mean().rename_axis('Owner').reset_index(name=new_col_name).astype({ new_col_name: 'float64' })
    temp_df = main_df.merge(temp_col, left_on='Owner', right_on='Owner', how='left')
    return temp_df

In [207]:
aggregate_owners = add_avg_vals(aggregate_owners, 'AverageGHGEmissionsIntensity', 'GHGEmissionsIntensity')
aggregate_owners.head()

,Owner,BuildingsOwned,TotalSquareFootage,TotalGHGEmissions__metric_tons_,TotalElectricity_kBtu_,TotalSteamUse_kBtu,TotalNaturalGas_kBtu,TotalOtherFuelUse_kBtu,AverageGHGEmissionsIntensity
0,CITY OF SEATTLE,175,17038324,16357,501287744,6826192,247129120,0,1.172674
1,UNIVERSITY OF WASHINGTON,43,18445870,39712,679730875,366595771,106707401,0,4.756098
2,STATE OF WASHINGTON,17,2768746,3614,88426130,0,59208493,0,2.112500
3,KING COUNTY,16,4355059,22535,255701627,16515088,372904431,0,1.750000
4,OVERLOOK MAGNOLIA APARTMENTS LLC,11,388815,52,9911319,0,0,0,0.127273


In [208]:
aggregate_owners = add_avg_vals(aggregate_owners, 'AverageSiteEUI_kBtu_sf', 'SiteEUI_kBtu_sf_')
aggregate_owners.head()

,Owner,BuildingsOwned,TotalSquareFootage,TotalGHGEmissions__metric_tons_,TotalElectricity_kBtu_,TotalSteamUse_kBtu,TotalNaturalGas_kBtu,TotalOtherFuelUse_kBtu,AverageGHGEmissionsIntensity,AverageSiteEUI_kBtu_sf
0,CITY OF SEATTLE,175,17038324,16357,501287744,6826192,247129120,0,1.172674,49.970175
1,UNIVERSITY OF WASHINGTON,43,18445870,39712,679730875,366595771,106707401,0,4.756098,147.265000
2,STATE OF WASHINGTON,17,2768746,3614,88426130,0,59208493,0,2.112500,71.162500
3,KING COUNTY,16,4355059,22535,255701627,16515088,372904431,0,1.750000,66.943750
4,OVERLOOK MAGNOLIA APARTMENTS LLC,11,388815,52,9911319,0,0,0,0.127273,25.418182


In [209]:
aggregate_owners = add_avg_vals(aggregate_owners, 'AverageSourceEUI_kBtu_sf', 'SourceEUI_kBtu_sf_')
aggregate_owners.head()

,Owner,BuildingsOwned,TotalSquareFootage,TotalGHGEmissions__metric_tons_,TotalElectricity_kBtu_,TotalSteamUse_kBtu,TotalNaturalGas_kBtu,TotalOtherFuelUse_kBtu,AverageGHGEmissionsIntensity,AverageSiteEUI_kBtu_sf,AverageSourceEUI_kBtu_sf
0,CITY OF SEATTLE,175,17038324,16357,501287744,6826192,247129120,0,1.172674,49.970175,108.376608
1,UNIVERSITY OF WASHINGTON,43,18445870,39712,679730875,366595771,106707401,0,4.756098,147.265000,321.932500
2,STATE OF WASHINGTON,17,2768746,3614,88426130,0,59208493,0,2.112500,71.162500,134.625000
3,KING COUNTY,16,4355059,22535,255701627,16515088,372904431,0,1.750000,66.943750,137.293750
4,OVERLOOK MAGNOLIA APARTMENTS LLC,11,388815,52,9911319,0,0,0,0.127273,25.418182,71.145455


In [210]:

aggregate_owners = add_avg_vals(aggregate_owners, 'AverageENERGYSTARScore', 'ENERGYSTARScore')
aggregate_owners.head()

,Owner,BuildingsOwned,TotalSquareFootage,TotalGHGEmissions__metric_tons_,TotalElectricity_kBtu_,TotalSteamUse_kBtu,TotalNaturalGas_kBtu,TotalOtherFuelUse_kBtu,AverageGHGEmissionsIntensity,AverageSiteEUI_kBtu_sf,AverageSourceEUI_kBtu_sf,AverageENERGYSTARScore
0,CITY OF SEATTLE,175,17038324,16357,501287744,6826192,247129120,0,1.172674,49.970175,108.376608,72.747899
1,UNIVERSITY OF WASHINGTON,43,18445870,39712,679730875,366595771,106707401,0,4.756098,147.265000,321.932500,61.818182
2,STATE OF WASHINGTON,17,2768746,3614,88426130,0,59208493,0,2.112500,71.162500,134.625000,55.375000
3,KING COUNTY,16,4355059,22535,255701627,16515088,372904431,0,1.750000,66.943750,137.293750,68.083333
4,OVERLOOK MAGNOLIA APARTMENTS LLC,11,388815,52,9911319,0,0,0,0.127273,25.418182,71.145455,74.545455


In [211]:
aggregate_owners.to_csv('landlords_with_total_energy_use_2_16_23.csv')

## Top-ranked polluters

Now that we have the aggregate numbers for each landlord, let's see who's causing the most damage to the environment!

We had some questions about Amazon and Vulcan as landlords. Checking which buildings they own.

In [215]:
aggregate_owners[aggregate_owners['Owner'].str.contains('VULCAN')]

,Owner,BuildingsOwned,TotalSquareFootage,TotalGHGEmissions__metric_tons_,TotalElectricity_kBtu_,TotalSteamUse_kBtu,TotalNaturalGas_kBtu,TotalOtherFuelUse_kBtu,AverageGHGEmissionsIntensity,AverageSiteEUI_kBtu_sf,AverageSourceEUI_kBtu_sf,AverageENERGYSTARScore


In [214]:
aggregate_owners[aggregate_owners['Owner'].str.contains('AMAZON')]

,Owner,BuildingsOwned,TotalSquareFootage,TotalGHGEmissions__metric_tons_,TotalElectricity_kBtu_,TotalSteamUse_kBtu,TotalNaturalGas_kBtu,TotalOtherFuelUse_kBtu,AverageGHGEmissionsIntensity,AverageSiteEUI_kBtu_sf,AverageSourceEUI_kBtu_sf,AverageENERGYSTARScore


We know that Vulcan and Amazon own buildings in the city, but they're not in our dataset. That is because they're owned by subsidiary companies. This will require some digging to figure out.

I don't know of any databases where you can look up a company's parent company. If we can't find it in a database, we'll need to go building by building based on public knowledge about each company's portfolio. E.g., [here](https://vulcanrealestate.com/Properties.aspx) is a list of properties owned by Vulcan. It is probably not complete.

### Biggest landowners

In [226]:
biggest_landowners = aggregate_owners.sort_values(by='BuildingsOwned', ascending=False)[:20]
biggest_landowners

,Owner,BuildingsOwned,TotalSquareFootage,TotalGHGEmissions__metric_tons_,TotalElectricity_kBtu_,TotalSteamUse_kBtu,TotalNaturalGas_kBtu,TotalOtherFuelUse_kBtu,AverageGHGEmissionsIntensity,AverageSiteEUI_kBtu_sf,AverageSourceEUI_kBtu_sf,AverageENERGYSTARScore
0,CITY OF SEATTLE,175,17038324,16357,501287744,6826192,247129120,0,1.172674,49.970175,108.376608,72.747899
1,UNIVERSITY OF WASHINGTON,43,18445870,39712,679730875,366595771,106707401,0,4.756098,147.265000,321.932500,61.818182
2,STATE OF WASHINGTON,17,2768746,3614,88426130,0,59208493,0,2.112500,71.162500,134.625000,55.375000
3,KING COUNTY,16,4355059,22535,255701627,16515088,372904431,0,1.750000,66.943750,137.293750,68.083333
4,OVERLOOK MAGNOLIA APARTMENTS LLC,11,388815,52,9911319,0,0,0,0.127273,25.418182,71.145455,74.545455
5,CATHOLIC ARCHDIOCESE OF SEATTLE,11,625312,446,10085358,839547,6078878,0,1.245455,56.020000,117.250000,71.666667
6,CSHV NWCP SEATTLE LLC,10,607134,357,6497555,0,6082306,0,0.420000,17.940000,38.350000,64.100000
7,QWEST CORPORATION,10,932134,877,22242960,0,14302433,0,1.050000,50.500000,108.922222,57.857143
8,PUBLIC STORAGE,10,1302736,112,6498748,0,1473800,0,0.157143,10.214286,24.642857,NaN
9,SELIG HOLDINGS COMPANY LLC,9,1688556,675,92368368,0,3469057,0,0.500000,56.633333,151.955556,76.777778


In [227]:
biggest_landowners.to_csv('biggest_landowners.csv')

### Biggest polluters

In [228]:
biggest_polluters = aggregate_owners.sort_values(by='TotalGHGEmissions__metric_tons_', ascending=False)[:20]
biggest_polluters

,Owner,BuildingsOwned,TotalSquareFootage,TotalGHGEmissions__metric_tons_,TotalElectricity_kBtu_,TotalSteamUse_kBtu,TotalNaturalGas_kBtu,TotalOtherFuelUse_kBtu,AverageGHGEmissionsIntensity,AverageSiteEUI_kBtu_sf,AverageSourceEUI_kBtu_sf,AverageENERGYSTARScore
1,UNIVERSITY OF WASHINGTON,43,18445870,39712,679730875,366595771,106707401,0,4.756098,147.265000,321.932500,61.818182
3,KING COUNTY,16,4355059,22535,255701627,16515088,372904431,0,1.750000,66.943750,137.293750,68.083333
0,CITY OF SEATTLE,175,17038324,16357,501287744,6826192,247129120,0,1.172674,49.970175,108.376608,72.747899
152,SWEDISH HOSPITAL MEDICAL CENTE,2,1659904,11941,133113224,125030832,16108152,0,5.600000,143.250000,274.850000,40.500000
896,JAMES TOWER LLC,1,597519,4805,56050971,0,84872338,0,8.000000,234.200000,409.000000,47.000000
1706,KOLL LAKE UNION ASSOC L P,1,527289,4524,60154140,0,79160244,0,8.600000,259.900000,469.400000,NaN
2,STATE OF WASHINGTON,17,2768746,3614,88426130,0,59208493,0,2.112500,71.162500,134.625000,55.375000
14,SEATTLE UNIVERSITY,6,2624288,3019,49391827,0,51904014,0,0.533333,27.880000,57.980000,93.000000
17,SEATTLE PACIFIC UNIVERSITY,5,753801,2150,21990177,0,38279666,0,1.500000,48.560000,90.820000,94.000000
1445,CHILDREN'S HOSPITAL & REGIONAL MEDICAL CENTER,1,230638,1927,27185311,0,33574922,0,8.400000,243.000000,445.500000,NaN


In [229]:
biggest_polluters.to_csv('biggest_polluters.csv')

### Biggest users of each energy type

In [236]:
aggregate_owners['TotalOtherFuelUse_kBtu'].unique()

array([0])

In [251]:
# not using other fuel use because they're all zero
energy_type_use = aggregate_owners[['Owner', 'TotalGHGEmissions__metric_tons_', 'TotalNaturalGas_kBtu', 'TotalElectricity_kBtu_', 'TotalSteamUse_kBtu']]
energy_type_use.sort_values(by='TotalGHGEmissions__metric_tons_', ascending=False)[:20]

,Owner,TotalGHGEmissions__metric_tons_,TotalNaturalGas_kBtu,TotalElectricity_kBtu_,TotalSteamUse_kBtu
1,UNIVERSITY OF WASHINGTON,39712,106707401,679730875,366595771
3,KING COUNTY,22535,372904431,255701627,16515088
0,CITY OF SEATTLE,16357,247129120,501287744,6826192
152,SWEDISH HOSPITAL MEDICAL CENTE,11941,16108152,133113224,125030832
896,JAMES TOWER LLC,4805,84872338,56050971,0
1706,KOLL LAKE UNION ASSOC L P,4524,79160244,60154140,0
2,STATE OF WASHINGTON,3614,59208493,88426130,0
14,SEATTLE UNIVERSITY,3019,51904014,49391827,0
17,SEATTLE PACIFIC UNIVERSITY,2150,38279666,21990177,0
1445,CHILDREN'S HOSPITAL & REGIONAL MEDICAL CENTER,1927,33574922,27185311,0


In [248]:
biggest_gas_users = energy_type_use.sort_values(by='TotalNaturalGas_kBtu', ascending=False)[:20]
biggest_gas_users

,Owner,TotalGHGEmissions__metric_tons_,TotalNaturalGas_kBtu,TotalElectricity_kBtu_,TotalSteamUse_kBtu
3,KING COUNTY,22535,372904431,255701627,16515088
0,CITY OF SEATTLE,16357,247129120,501287744,6826192
1,UNIVERSITY OF WASHINGTON,39712,106707401,679730875,366595771
896,JAMES TOWER LLC,4805,84872338,56050971,0
1706,KOLL LAKE UNION ASSOC L P,4524,79160244,60154140,0
2,STATE OF WASHINGTON,3614,59208493,88426130,0
14,SEATTLE UNIVERSITY,3019,51904014,49391827,0
17,SEATTLE PACIFIC UNIVERSITY,2150,38279666,21990177,0
1445,CHILDREN'S HOSPITAL & REGIONAL MEDICAL CENTER,1927,33574922,27185311,0
1602,CITY INVESTORS XII L L C,1707,28404272,37437965,0


In [239]:
biggest_gas_users.to_csv('biggest_gas_users.csv')

In [247]:
biggest_electric_users = energy_type_use.sort_values(by='TotalElectricity_kBtu_', ascending=False)[:20]
biggest_electric_users

,Owner,TotalGHGEmissions__metric_tons_,TotalNaturalGas_kBtu,TotalElectricity_kBtu_,TotalSteamUse_kBtu
1,UNIVERSITY OF WASHINGTON,39712,106707401,679730875,366595771
0,CITY OF SEATTLE,16357,247129120,501287744,6826192
310,2001 SIXTH LLC,1455,315739,270558198,0
3,KING COUNTY,22535,372904431,255701627,16515088
10,ACORN DEVELOPMENT LLC,1595,15879116,141486254,0
152,SWEDISH HOSPITAL MEDICAL CENTE,11941,16108152,133113224,125030832
1616,GI TC SEATTLE LLC,709,417207,129155774,0
9,SELIG HOLDINGS COMPANY LLC,675,3469057,92368368,0
2,STATE OF WASHINGTON,3614,59208493,88426130,0
187,2020 FIFTH HOLDINGS LLC,453,0,85314262,0


In [242]:
# Wow, what is 2001 6th Ave?
# It's the Westin Hotel
aggregate_owners[aggregate_owners['Owner'] == '2001 SIXTH LLC']

,Owner,BuildingsOwned,TotalSquareFootage,TotalGHGEmissions__metric_tons_,TotalElectricity_kBtu_,TotalSteamUse_kBtu,TotalNaturalGas_kBtu,TotalOtherFuelUse_kBtu,AverageGHGEmissionsIntensity,AverageSiteEUI_kBtu_sf,AverageSourceEUI_kBtu_sf,AverageENERGYSTARScore
310,2001 SIXTH LLC,1,429405,1455,270558198,0,315739,0,3.4,630.8,1765.0,NaN


So the Westin Hotel is the third largest electricity user--from one building! That's something to look at, along with hotels more generally.

In [243]:
biggest_electric_users.to_csv('biggest_electric_users.csv')

In [246]:
biggest_steam_users = energy_type_use.sort_values(by='TotalSteamUse_kBtu', ascending=False)[:20]
biggest_steam_users

,Owner,TotalGHGEmissions__metric_tons_,TotalNaturalGas_kBtu,TotalElectricity_kBtu_,TotalSteamUse_kBtu
1,UNIVERSITY OF WASHINGTON,39712,106707401,679730875,366595771
152,SWEDISH HOSPITAL MEDICAL CENTE,11941,16108152,133113224,125030832
1237,HART FIRST HILL LLC,1439,0,11707490,16594533
3,KING COUNTY,22535,372904431,255701627,16515088
1376,WASHINGTON STATE CONVENTION CENTER,1471,1251147,30223520,14987953
1383,FIRST HILL/EIGHTH AVENUE LLC,1373,1272685,18818364,14524050
1452,MILNER VENTURES LLC,1061,1078170,8436545,11553532
1451,1415 5TH AVENUE SEATTLE LLC,1024,905623,12954470,10933061
1403,SEATTLE HOST HOTEL COMPANY L L C,1043,895500,25517379,10359896
1444,1301 SECOND AVE LLC,997,884053,22289102,10024828


In [245]:
biggest_steam_users.to_csv('biggest_steam_users.csv')

We have several things to pull out of this energy use rankings:
    - UW, City of Seattle, State of WA, and King County are consistently in the top polluters for each energy type (and energy usage overall)
    - Hotels are huge energy users, and we haven't done much outreach/strategizing about them
    - Hospitals are big polluters; we knew this already
    - Even without Amazon/Vulcan, we still have landlords who are big polluters that we can target (side note: current opposition research tells us that Amazon/Vulcan won't challenge BPS because it's more trouble than it's worth)

In [256]:
# percentage of gas/electric/steam used in govt buildings
# vs all use

total_city_use_of_energy = pandas.DataFrame({
    'TotalGHGEmissions__metric_tons_': [buildings_dataframe['TotalGHGEmissions__metric_tons_'].sum()],
    'TotalNaturalGas_kBtu': [buildings_dataframe['NaturalGas_kBtu_'].sum()],
    'TotalElectricity_kBtu_': [buildings_dataframe['Electricity_kBtu_'].sum()],
    'TotalSteamUse_kBtu': [buildings_dataframe['SteamUse_kBtu_'].sum()]
})

total_city_use_of_energy

,TotalGHGEmissions__metric_tons_,TotalNaturalGas_kBtu,TotalElectricity_kBtu_,TotalSteamUse_kBtu
0,401082.3,4716043148,11658750649,1067607092


In [257]:
govt_owners = ['UNIVERSITY OF WASHINGTON', 'CITY OF SEATTLE', 'PORT OF SEATTLE', 'KING COUNTY', 'STATE OF WASHINGTON']

In [ ]:
print([aggregate_owners[aggregate_owners['Owner'] == owner]['TotalGHGEmissions__metric_tons_'] for owner in govt_owners])

In [287]:
ghge = [aggregate_owners[aggregate_owners['Owner'] == owner]['TotalGHGEmissions__metric_tons_'] for owner in govt_owners]
govt_energy_use = pandas.DataFrame({
    'Owner': govt_owners,
    'TotalGHGEmissions__metric_tons_': [1, 2, 3, 4,5]
})

In [288]:
govt_energy_use.head()

In [275]:
# hospital/landowners
# hotel energy use
# pie chart of UW energy use
# bar chart of biggest polluters